# Проект для «Викишоп»

Интернет-магазин «Викишоп» запускает новый сервис. Теперь пользователи могут редактировать и дополнять описания товаров, как в вики-сообществах. То есть клиенты предлагают свои правки и комментируют изменения других. Магазину нужен инструмент, который будет искать токсичные комментарии и отправлять их на модерацию. 

Обучите модель классифицировать комментарии на позитивные и негативные. В вашем распоряжении набор данных с разметкой о токсичности правок.

Постройте модель со значением метрики качества *F1* не меньше 0.75. 

**Инструкция по выполнению проекта**

1. Загрузите и подготовьте данные.
2. Обучите разные модели. 
3. Сделайте выводы.


**Описание данных**

Данные находятся в файле `toxic_comments.csv`. Столбец *text* в нём содержит текст комментария, а *toxic* — целевой признак.

<h1>Содержание<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Подготовка" data-toc-modified-id="Подготовка-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Подготовка</a></span></li><li><span><a href="#Обучение" data-toc-modified-id="Обучение-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Обучение</a></span><ul class="toc-item"><li><span><a href="#DecisionTree" data-toc-modified-id="DecisionTree-2.1"><span class="toc-item-num">2.1&nbsp;&nbsp;</span>DecisionTree</a></span></li><li><span><a href="#RandomForest" data-toc-modified-id="RandomForest-2.2"><span class="toc-item-num">2.2&nbsp;&nbsp;</span>RandomForest</a></span></li><li><span><a href="#LogisticRegression" data-toc-modified-id="LogisticRegression-2.3"><span class="toc-item-num">2.3&nbsp;&nbsp;</span>LogisticRegression</a></span></li><li><span><a href="#LogisticRegression-with-Pipeline" data-toc-modified-id="LogisticRegression-with-Pipeline-2.4"><span class="toc-item-num">2.4&nbsp;&nbsp;</span>LogisticRegression with Pipeline</a></span></li></ul></li><li><span><a href="#Выводы" data-toc-modified-id="Выводы-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Выводы</a></span></li></ul></div>

## Подготовка

In [1]:
import pandas as pd
import numpy as np
import nltk
import re
import matplotlib.pyplot as plt
from tqdm import tqdm

from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import f1_score, make_scorer
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer 

import nltk
from nltk.corpus import stopwords as nltk_stopwords
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer 

from pymystem3 import Mystem

nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('punkt')

from sklearn.pipeline import Pipeline

import warnings
warnings.filterwarnings("ignore")

RND=12345

[nltk_data] Downloading package stopwords to /home/jovyan/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /home/jovyan/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to /home/jovyan/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [2]:
df=pd.read_csv('/datasets/toxic_comments.csv')

In [3]:
def fun_date(data):
    display ('**Общая информация датафрейма**')
    display(data.info())
    print(' ')
    display('**Просмотр статистических данных**')
    display(data.describe(include= 'all').T)
    print(' ')
    display('**Вывод первых пяти строк датафрейма**')
    display(data.head())
    print(' ')
    display('**Вывод количесва дубликатов датафрейма**')
    display(data.duplicated().sum())
    print(' ')
    display('**Вывод количесва пропущеных значений датафрейма**')
    display(data.isnull().sum())

    
display(fun_date(df))

'**Общая информация датафрейма**'

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 159571 entries, 0 to 159570
Data columns (total 2 columns):
 #   Column  Non-Null Count   Dtype 
---  ------  --------------   ----- 
 0   text    159571 non-null  object
 1   toxic   159571 non-null  int64 
dtypes: int64(1), object(1)
memory usage: 2.4+ MB


None

'**Просмотр статистических данных**'

,count,unique,top,freq,mean,std,min,25%,50%,75%,max
text,159571,159571,The Zafarnāma /zəfərnɑːmɑː/ (Punjabi: ਜ਼ਫ਼ਰਨਾਮ...,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN
toxic,159571.0,NaN,NaN,NaN,0.101679,0.302226,0.0,0.0,0.0,0.0,1.0


...

'**Вывод количесва дубликатов датафрейма**'

0

'**Вывод количесва пропущеных значений датафрейма**'

text     0
toxic    0
dtype: int64

None

In [4]:
m = Mystem() 
corpus = df['text'].values.astype('U')

def lemmatize(text):
    text=text.lower()
    lemm_list = m.lemmatize(text)
    lemm_text = " ".join(lemm_list)
    clear_text = re.sub(r'[^a-zA-Z]', ' ', lemm_text)
    return " ".join(clear_text.split())

In [5]:
#lemmatizer = WordNetLemmatizer()

#def lemmatize(text):
    #text=text.lower()
    #text = nltk.word_tokenize(text)
    #lemm_list = ' '.join([lemmatizer.lemmatize(w) for w in text])
    #return lemm_list

In [6]:
df['text']=df['text'].apply(lemmatize)

In [7]:
nltk.download('stopwords')
stopwords = set(nltk_stopwords.words('english'))
count_tf_idf = TfidfVectorizer(stop_words=stopwords)

[nltk_data] Downloading package stopwords to /home/jovyan/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [8]:
target = df['toxic']
features = df.drop(['toxic'], axis=1)

x_train, x_test, y_train, y_test = train_test_split(features, target, test_size=0.2, random_state=RND)

In [9]:
x_train = count_tf_idf.fit_transform(x_train['text'])
x_test= count_tf_idf.transform(x_test['text'])

In [10]:
df['toxic'].value_counts()

0    143346
1     16225
Name: toxic, dtype: int64

Т о , мы предобработали данные , также увидели , что классы не сбалансированы , поэтому будем использовать class_weight="balanced"

## Обучение

### DecisionTree

In [11]:
model = DecisionTreeClassifier(random_state=RND, class_weight="balanced")
params = {
   'criterion':['gini', 'entropy'],        
   'max_depth':[5,10,15]
}
tree_gs = GridSearchCV(model, params, cv=3, scoring='f1', verbose=True).fit(x_train, y_train)
print(tree_gs.best_params_)

Fitting 3 folds for each of 6 candidates, totalling 18 fits
{'criterion': 'entropy', 'max_depth': 15}


In [12]:
model=DecisionTreeClassifier(random_state = RND,class_weight="balanced", criterion= 'entropy', max_depth=15)
model.fit(x_train, y_train)

DecisionTreeClassifier(class_weight='balanced', criterion='entropy',
                       max_depth=15, random_state=12345)

In [13]:
print("Best f1 score RF:",f1_score(y_test,model.predict(x_test)))

Best f1 score RF: 0.5968278832974349


### RandomForest

In [14]:
model1 = RandomForestClassifier(random_state=RND, class_weight="balanced")
grid = {"max_depth":range(8,10), 'n_estimators':  range(98,100)}


grs=GridSearchCV( model1, param_grid =grid, cv=3, scoring='f1', verbose=True)
grs.fit(x_train, y_train)
print(grs.best_params_)

Fitting 3 folds for each of 4 candidates, totalling 12 fits
{'max_depth': 9, 'n_estimators': 98}


In [15]:
%%time
model1=RandomForestClassifier(random_state=RND, class_weight="balanced",max_depth= 9, n_estimators=98)
model1.fit(x_train, y_train)

CPU times: user 3.31 s, sys: 31.7 ms, total: 3.34 s
Wall time: 3.35 s


RandomForestClassifier(class_weight='balanced', max_depth=9, n_estimators=98,
                       random_state=12345)

In [16]:
print("Best f1 score RF:",f1_score(y_test, model1.predict(x_test)))

Best f1 score RF: 0.3630681078265399


### LogisticRegression

In [17]:
model2= LogisticRegression(random_state=RND, class_weight="balanced")
grid2 = GridSearchCV(model2, {"C":[0.9], "max_iter": [50,90]},  scoring='f1', verbose=True)

grs2=grid2.fit(x_train, y_train)
grs2.best_params_

Fitting 5 folds for each of 2 candidates, totalling 10 fits


{'C': 0.9, 'max_iter': 90}

In [18]:
print("Best f1 score LR:",f1_score(y_test, grs2.predict(x_test)))

Best f1 score LR: 0.7529251700680272


In [19]:
%%time

model2=LogisticRegression(C=0.9, max_iter = 90, random_state=RND, class_weight="balanced").fit(x_train, y_train)

CPU times: user 20.7 s, sys: 24.9 s, total: 45.6 s
Wall time: 45.7 s


In [20]:
print("Best f1 score LR:",f1_score(y_test, model2.predict(x_test)))

Best f1 score LR: 0.7529251700680272


### LogisticRegression with Pipeline

In [21]:
pipe_lr = Pipeline([('clf', LogisticRegression(random_state=RND))])
gs3= GridSearchCV(estimator=pipe_lr,
            param_grid=[{'clf__penalty': ['l1', 'l2'],
                         'clf__solver': ['liblinear']}] ,
            scoring='f1',
            cv=10) 

In [22]:
%%time

gs3.fit(x_train, y_train)

CPU times: user 55.8 s, sys: 40.9 s, total: 1min 36s
Wall time: 1min 37s


GridSearchCV(cv=10,
             estimator=Pipeline(steps=[('clf',
                                        LogisticRegression(random_state=12345))]),
             param_grid=[{'clf__penalty': ['l1', 'l2'],
                          'clf__solver': ['liblinear']}],
             scoring='f1')

In [23]:
gs3.best_params_

{'clf__penalty': 'l1', 'clf__solver': 'liblinear'}

In [24]:
print("Best f1 score LR PIP:",f1_score(y_test, gs3.predict(x_test)))

Best f1 score LR PIP: 0.772194011556645


## Выводы

Итак, мы предобработали данные, разбили данные на выборки, обучили 3 разных модели и подобрали гиперпараметры, определили лучшую модель по качеству.

Лучшее качество модели также у LogisticRegression, где F=0.7529 (LogisticRegression с Pipeline F=0,77), поэтому  Магазину "Викишоп" больше подходит модель LogisticRegression, которая будет искать токсичные комментарии и отправлять их на модерацию.

